# Exercise 15: Power Analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below. 

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill. 

`c` is reading comprehension, and `x` is word reading experience.

In [1]:
sample_size = 100 # How many children in data set? 
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0. 
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
      # WRITE YOUR CODE HERE
     
    r <- runif(sample_size, age_lo, age_hi)
    output_x = data.frame()
    output_c = data.frame()
    mean_r  <- mean(r) #generate mean(r)
    
    for (item in 1:sample_size)
        #x = (beta_xa + beta_x0 + rnorm(1, mean_x, sd_x))
        output_x = rbind(output_x, (beta_xa + beta_x0 + rnorm(1, mean = 0, sd_x)))
    
    for (item in 1:sample_size)
        #x = (beta_xa + beta_x0 + rnorm(1, mean_x, sd_x))
        output_c = rbind(output_c, (beta_ca + beta_cx + beta_c0 + rnorm(1, mean = 0, sd_c)))
    
    output = data.frame("age" = r, output_x, output_c)
    colnames(output)[2] ="x"
    colnames(output)[3] ="c"
    
    
      return(output)
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,143.1033,29.206743,113.269541
2,124.8551,6.325374,-1.054559
3,197.0996,177.386256,9.065584
4,157.7952,74.189848,21.320028
5,107.7914,-16.360315,102.425175
6,163.0641,-63.302769,44.499107


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data. 

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously. 

The following line confuses me, which data is it referring to?? :

Print this function's output for the data we simulated previously.

In [2]:
# WRITE YOUR CODE HERE
library(mediation)

#fitM <- lm(x ~ age,     data=dat) #Step 1: IV on M, age predicting x
#fitY <- lm(c ~ x + age, data=dat) #Step 2: IV and M on DV, x and age predicting c

#fitMed <- mediate(fitM, fitY, treat= "age", mediator= "c")
#summary(fitMed)

# a function for running a t-test and extracting p-value
run_analysis <- function(data) {
    fitM <- lm(x ~ age,    data=dat) #Step 1: IV on M, age predicting x
    fitY <- lm(c ~ x + age,    data=dat) #Step 2: IV and M on DV, x and age predicting c
    fit <- mediate(fitM, fitY, treat= "age", mediator= "c")  
    
    output  <- c(fit$d0, fit$d0.p, fit$z0, fit$z0.p)
    print(summary(fit))
    return(output)
    #return(summary(fit))
    #return(c(fit$estimate[1] - fit$estimate[2],fit$stderr,fit$p.value)) # this outputs the effect size, standard error, and significance for quality metrics 
}

rad  <- run_analysis(dat)
print(head(rad))

Warning message:
"package 'mediation' was built under R version 4.2.3"
Loading required package: MASS

Loading required package: Matrix

Loading required package: mvtnorm

Loading required package: sandwich

Warning message:
"package 'sandwich' was built under R version 4.2.3"
mediation: Causal Mediation Analysis
Version: 4.5.0





Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00     1.0
ADE               0.129       -0.271         0.52     0.5
Total Effect      0.129       -0.271         0.52     0.5
Prop. Mediated    0.000        0.000         0.00     1.0

Sample Size Used: 100 


Simulations: 1000 

[1] 0.000000 1.000000 0.128671 0.502000


---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [3]:
#num_simulations = 2
#alpha = .05

repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)
    
    #print(paste("Sample Size:", sample_size, sep = ''))
        
    # Start simulating
    for (i in 1:num_simulations) {
      sim_dat  <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      run_analysis  <- run_analysis(sim_dat)
      simouts[i, ]  <- run_analysis
      #print(paste("Simulation #", i, sep = ' ')) #debug statement
    }
    
    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    #cvg <- mean(simouts[,3] <= alpha)
    ACME_cov = mean(simouts[,1] <= alpha) # D0 = ACME/Average causal mediated effect
    ADE_cov = mean(simouts[,3] <= alpha) # z0 = ADE/Average direct effects

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
    #return(simouts)
}

#print(cov(a, b, method = "spearman")) 

repeat_analysis(num_simulations = 5, alpha = 0.05, sample_size = 50, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
#head(repeat_analysis)


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE               0.149       -0.234         0.55    0.46
Total Effect      0.149       -0.234         0.55    0.46
Prop. Mediated    0.000        0.000         0.00    1.00

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00     1.0
ADE               0.138       -0.258         0.55     0.5
Total Effect      0.138       -0.258         0.55     0.5
Prop. Mediated    0.000        0.000         0.00     1.0

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE           

$ACME_cov
[1] 1

$ADE_cov
[1] 0

In [4]:
data("Income")

## find and some rules (we only use 5 rules here) and calculate coverage
rules <- apriori(Income)[1:5]
quality(rules) <- cbind(quality(rules), coverage = coverage(rules))

inspect(rules)

Warning message in data("Income"):
"data set 'Income' not found"


ERROR: Error in apriori(Income): could not find function "apriori"


In [ ]:
#simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)

#simouts[1, ]  <- c(2, 3, 4, 4)
#simouts

#print(paste("Simulation #", 1, sep = ''))

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01. 

In [ ]:
num_simulations = 10
alpha = .01

repeat_analysis(num_simulations, alpha, sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run. 

In [ ]:
# WRITE YOUR CODE HERE
library(purrr)

num_simulations = 2
alpha = .01

named_vector  <- c(1, 2, 3, 4)

sample_size = NA

#Failed for loop
for (item in named_vector)
    sample_size  <-  item
    print(item)
    repeat_analysis(num_simulations, alpha, item, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

In [ ]:
sample_size = 100

x = 20

#repeat_analysis(num_simulations, alpha, sample_size = x, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

list.map = list(50, 75, 100)

for (item in list.map)
    x = item
    run_repeat_analysis  <- repeat_analysis(num_simulations = 5, alpha = 0.01, sample_size = x, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
    print(paste("Item", item, "finished", sep = ' '))

Print your results. 

In [5]:
# TERRIBLE BORING UGLY SOLUTION THAT I HATE
sample_size = 50

repeat_analysis(num_simulations = 100, alpha = 0.01, sample_size = 50, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
repeat_analysis(num_simulations = 100, alpha = 0.01, sample_size = 75, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
repeat_analysis(num_simulations = 100, alpha = 0.01, sample_size = 100, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
repeat_analysis(num_simulations = 100, alpha = 0.01, sample_size = 125, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
repeat_analysis(num_simulations = 100, alpha = 0.01, sample_size = 150, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE               0.143       -0.279         0.56    0.45
Total Effect      0.143       -0.279         0.56    0.45
Prop. Mediated    0.000        0.000         0.00    1.00

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE               0.141       -0.293         0.56    0.51
Total Effect      0.141       -0.293         0.56    0.51
Prop. Mediated    0.000        0.000         0.00    1.00

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE           

$ACME_cov
[1] 1

$ADE_cov
[1] 0


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE               0.133       -0.274         0.55    0.51
Total Effect      0.133       -0.274         0.55    0.51
Prop. Mediated    0.000        0.000         0.00    1.00

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE               0.137       -0.277         0.54    0.49
Total Effect      0.137       -0.277         0.54    0.49
Prop. Mediated    0.000        0.000         0.00    1.00

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE           

$ACME_cov
[1] 1

$ADE_cov
[1] 0


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00     1.0
ADE               0.141       -0.286         0.55     0.5
Total Effect      0.141       -0.286         0.55     0.5
Prop. Mediated    0.000        0.000         0.00     1.0

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE               0.148       -0.256         0.55    0.48
Total Effect      0.148       -0.256         0.55    0.48
Prop. Mediated    0.000        0.000         0.00    1.00

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE           

$ACME_cov
[1] 1

$ADE_cov
[1] 0


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE               0.133       -0.307         0.56    0.51
Total Effect      0.133       -0.307         0.56    0.51
Prop. Mediated    0.000        0.000         0.00    1.00

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE               0.137       -0.263         0.54    0.52
Total Effect      0.137       -0.263         0.54    0.52
Prop. Mediated    0.000        0.000         0.00    1.00

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE           

$ACME_cov
[1] 1

$ADE_cov
[1] 0


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE               0.142       -0.242         0.56    0.49
Total Effect      0.142       -0.242         0.56    0.49
Prop. Mediated    0.000        0.000         0.00    1.00

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE               0.156       -0.262         0.57    0.45
Total Effect      0.156       -0.262         0.57    0.45
Prop. Mediated    0.000        0.000         0.00    1.00

Sample Size Used: 100 


Simulations: 1000 


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value
ACME              0.000        0.000         0.00    1.00
ADE           

$ACME_cov
[1] 1

$ADE_cov
[1] 0

## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above? 

> *Write your response here.* 
> 

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *Write your response here.* 

**DUE:** 5pm EST, April 5, 2023

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*